# Setup of ClickHouse columnar DB for analytics

In [4]:
#custom cell command, run it before runing code below
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [5]:
#define variables

user_name="parkman_user"
user_pass="parkman_user_pass"



Write config files, defining users for db (users.xml) and server config (config.xml)

In [6]:
%%writefile ./config.xml
<clickhouse replace="true">
    <logger>
        <level>debug</level>
        <log>/var/log/clickhouse-server/clickhouse-server.log</log>
        <errorlog>/var/log/clickhouse-server/clickhouse-server.err.log</errorlog>
        <size>1000M</size>
        <count>3</count>
    </logger>
    <display_name>clickhouse_analytics</display_name>
    <listen_host>0.0.0.0</listen_host>
    <http_port>8123</http_port>
    <tcp_port>9000</tcp_port>
    <user_directories>
        <users_xml>
            <path>users.xml</path>
        </users_xml>
        <local_directory>
            <path>/var/lib/clickhouse/access/</path>
        </local_directory>
    </user_directories>
</clickhouse>

Overwriting ./config.xml


In [19]:
%%writefile ./users.xml
<?xml version="1.0"?>
<clickhouse replace="true">
    <profiles>
        <default>
            <max_memory_usage>10000000000</max_memory_usage>
            <use_uncompressed_cache>0</use_uncompressed_cache>
            <load_balancing>in_order</load_balancing>
            <log_queries>1</log_queries>
        </default>
    </profiles>
    <users>
        <default > <!--NOT ALLOWED IN PRODUCTION-->
            <access_management>1</access_management>
            <profile>default</profile>
            <networks>
                <ip>::/0</ip>
            </networks>
            <quota>default</quota>
            <access_management>1</access_management>
            <named_collection_control>1</named_collection_control>
            <show_named_collections>1</show_named_collections>
            <show_named_collections_secrets>1</show_named_collections_secrets>
        </default>
    </users>
    <quotas>
        <default>
            <interval>
                <duration>3600</duration>
                <queries>0</queries>
                <errors>0</errors>
                <result_rows>0</result_rows>
                <read_rows>0</read_rows>
                <execution_time>0</execution_time>
            </interval>
        </default>
    </quotas>
</clickhouse>

Overwriting ./users.xml


Write initdb script which inits DB schema and defines users so that access to DB is limited

In [7]:
%%writetemplate ./scripts/clickhouse-init.sql






CREATE DATABASE IF NOT EXISTS parking_db;
CREATE TABLE IF NOT EXISTS parking_db.parking_analytics  (
    owner_id String,            -- ID of parking lot owner (from MongoDB)
    owner_full_name String,     -- Name + surname of owner (MongoDB)
    parking_lot_id String,      -- ID of parking lot (from TimescaleDB)
    parking_lot_name String,    -- Name of parking lot (MongoDB)
    parking_spot_number Int32,  -- Number of parking spots for the parking lot (from MongoDB)
    user_id String,             -- ID of user who parked (from TimescaleDB)
    user_full_name String,             -- Name + surname of user (MongoDB)
    entry_timestamp DateTime,   -- When user entered parking lot (from TimescaleDB)
    leaving_timestamp DateTime, -- When user left parking lot (from TimescaleDB)
    checkout_price Float64      -- How much user has to pay for parking (from TimescaleDB)
) 
ENGINE = MergeTree()
PARTITION BY (owner_id, toYYYYMM(entry_timestamp))  -- Composite partition by owner_id and year-month of entry_timestamp
ORDER BY (owner_id, owner_name, parking_lot_id, parking_lot_name, entry_timestamp, user_id, user_name);  -- Order by owner_id and name, parking_lot_id and name, entry_timestamp, and user_id

CREATE USER {user_name} IDENTIFIED BY '{user_pass}';
GRANT ALTER, SELECT ON parking_db.parking_analytics TO {user_name}; 
GRANT ALTER ON parking_db TO {user_name}; 




Write yaml for clickhouse service

In [12]:
%%writefile ../clickhouse.yml
version: '3.8'
services:
  clickhouse:
    image: clickhouse/clickhouse-server
    container_name: clickhouse
    hostname: clickhouse
    volumes:
      - ./ClickHouseDB/config.xml:/etc/clickhouse-server/config.d/config.xml
      - ./ClickHouseDB/users.xml:/etc/clickhouse-server/users.d/users.xml
      - clickhouse_db:/var/lib/clickhouse
      - clickhouse_logs:/var/log/clickhouse-server/
      - ./ClickHouseDB/scripts:/docker-entrypoint-initdb.d 
    ports:
      - "8123:8123" #HTTP port
      - "9999:9000" #Native Client port

    networks:
      - app-network

volumes:
  clickhouse_db: {}
  clickhouse_logs: {}

  

Overwriting ../clickhouse.yml
